In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command, Input, Output
from azure.ai.ml import dsl

# Let's login to configure your workspace and resource group.
credential = DefaultAzureCredential()

# Get a handle to the workspace. You can find the info on the workspace tab on ml.azure.com
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/daniel/repos/aml_demo/config.json


In [32]:
import os
os.mkdir("../components/train_yolo_from_notebook")

FileExistsError: [Errno 17] File exists: '../components/train_yolo_from_notebook'

In [ ]:
%%writefile ../components/train_yolo_from_notebook/train_yolo_model.py

import argparse
import os
from ultralytics import YOLO
import mlflow
import glob
import subprocess
import re

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_model", type=str, default="yolo11n.pt", help="Model to train")
    parser.add_argument("--data", type=str, default="../data/coco128.yaml", help="Data config file")
    parser.add_argument("--epochs", type=int, default=3, help="Number of epochs to train")
    parser.add_argument("--output_model", type=str, default="./outputs/model", help="Output model path")
    args = parser.parse_args()

    print("Model output path: ", args.output_model)
    # Create the output directory if it doesn't exist
    if not os.path.exists(args.output_model):
        os.makedirs(args.output_model)
        print("Output directory created:", args.output_model)
    #get the yaml file from the data folder:
    files_in_dir = os.listdir(args.data)
    print("Files in directory:", files_in_dir)
    yaml = glob.glob(os.path.join(args.data, "*.yaml"))
    print("YAML files:", yaml)
    #get the first yaml file
    yaml = yaml[0]
    print("YAML file:", yaml)
    
    #in the yaml file, replace the whole line after 'path: ' with the the path in 'args.data'
    with open(yaml, "r+") as f:
        data = f.read()
        #in data, find the line that starts with 'path: ' and replace it with 'path: args.data'
        data = re.sub(r'path: .*', f'path: {args.data}', data)
        #write the data back to the file
        f.write(data)
    
    # Train the model
    with mlflow.start_run() as run:
        mlflow.log_param("base_model", args.base_model)
        mlflow.log_param("data", yaml)
        mlflow.log_param("epochs", args.epochs)
        mlflow.log_param("output_model", args.output_model)

        # Train the model
        model = YOLO(args.base_model)
        results = model.train(data=yaml, epochs=args.epochs, project=args.output_model, name="model")
        # model.save(f"{args.output_model}/yolo_coco128.pt")
        # print("Model saved to", args.output_model)
    
if __name__ == "__main__":
    main()



Writing ../components/train_yolo_from_notebook/train_yolo_model.py


In [34]:
path="azureml://datastores/container_raw/paths/coco128"

In [ ]:
train_yolo_component = command(
    name="train_yolo_model",
    description="Train a YOLO model",
    environment="yolofromdocker:2",
    inputs={
        "base_model": Input(type="string", default="yolo11n.pt"),
        "model_name": Input(type="string", default="yolofromnotebook"),
        "data": Input(type="uri_folder"),
        "epochs": Input(type="integer", default=3),
    },
    outputs={
            "output_model": Output(
                type="custom_model",
                name="yolofromnotebookmodel",
                path=f"azureml://datastores/container_processed/paths/trained_model_from_notebook/yolofromnotebook")},
    code = "../components/train_yolo_from_notebook",
    command="python train_yolo_model.py --base_model ${{inputs.base_model}} --data ${{inputs.data}} --epochs ${{inputs.epochs}} --output_model ${{outputs.output_model}}",
)

In [42]:
@dsl.pipeline(compute = "bigcompute", description="Train a YOLO model", experiment_name="train_yolo_model")
def train_yolo_model_pipeline(base_model: str, data: Input, epochs: int):
    # Create a command job to train the YOLO model
    train_yolo_job = train_yolo_component(
        base_model=base_model,
        data=data,
        epochs=epochs,
    )

    return {"output_model": train_yolo_job.outputs.output_model}

In [43]:
train_yolo_model_pipeline_instance = train_yolo_model_pipeline(
    base_model="yolo11n.pt",
    data=Input(type="uri_folder", mode = "rw_mount", path="azureml://datastores/container_raw/paths/coco128"),
    epochs=1,
)

In [44]:
ml_client.jobs.create_or_update(train_yolo_model_pipeline_instance)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.CustomModelJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
train_yolo_model,icy_sprout_mq338t7g4p,pipeline,NotStarted,Link to Azure Machine Learning studio


In [179]:
all_models = ml_client.models.list()
for model in all_models:
    print(model.name, model.version, model.description)

azureml_cd0dbbbc-0ec0-47f2-9dd4-a9165cd62756_output_output_model None None
azureml_bc1cebe5-21a3-4e3f-8ccb-3e69fb52aebc_output_output_model None None
azureml_cdac7285-f9ea-4949-9564-c6823ba6d7aa_output_output_model None None
azureml_648b169a-43aa-435b-af0a-b13748980cf1_output_output_model None None
azureml_250e0f8b-fb29-4587-a18c-2c93cf793d39_output_output_model None None
azureml_944fe082-4ccd-4ce5-a847-a385bf0d3a69_output_output_model None None
azureml_cffa05b3-9256-4db3-a269-e21da146cb6a_output_output_model None None
azureml_78a4f16b-e4fc-4295-a48c-c0367be1ee8e_output_output_model None None
azureml_bc4192c9-c501-40fb-8dac-3665d4754cf1_output_output_model None None
yolomodel_small None None
yolomodel None None
catsanddogsmodel None None
sam2model None None
azureml_22eac4ff-e422-4116-b935-d35249f50370_output_output_model None None
azureml_8699aad9-81bf-4705-a24a-946fbbbb4090_output_output_model None None
azureml_200204b1-caf8-4918-8372-7bd93df0bb41_output_output_model None None
azureml